# Диалоговый бот

## Проверяем карту

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Установка окружения

In [2]:
pip install transformers==4.6.1

     |████████████████████████████████| 2.3MB 5.6MB/s 
     |████████████████████████████████| 901kB 20.0MB/s 
     |████████████████████████████████| 3.3MB 30.9MB/s 


In [ ]:
%%writefile setup.sh
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
cd apex
git reset --hard a651

pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Writing setup.sh


In [ ]:
# Нужно устанавливать, только если запускать на GPU, но это не обязательно. Ставится долго
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 8048 (delta 65), reused 93 (delta 41), pack-reused 7913
Receiving objects: 100% (8048/8048), 14.10 MiB | 28.10 MiB/s, done.
Resolving deltas: 100% (5466/5466), done.
HEAD is now at a651e2c sync-free Distributed LAMB + parameter reordering (#1055)
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-9q2ur2pj
Created temporary directory: /tmp/pip-req-tracker-x2pk0q2l
Created requirements tracker '/tmp/pip-req-tracker-x2pk0q2l'
Created temporary directory: /tmp/pip-install-lftw_mww
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-dbuo8vmi
  Added file

## Подключаем диск для загрузки модели

In [7]:
MODEL_DOWNLOAD_PATH='/content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/models'

In [8]:
# Обеспечиваем подгрузку данных и их хранение в каталоге ноутубка MyDrive/chats_emotions_and_voises/chat04_depression-therapist-chatbot
import os
from google.colab import drive
drive.mount('/content/gdrive')
if not os.path.exists(MODEL_DOWNLOAD_PATH):
    raise ValueError('Нет папки для хранения данных', MODEL_DOWNLOAD_PATH)
%ls $MODEL_DOWNLOAD_PATH

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
checkpoin60.zip  checkpoint47.zip  saved_model47000.zip  saved_model70000.zip


## Скачиваем модель модель

In [9]:
MODEL_PATH = '/content/saved_model'
LAST_MODEL_ZIP = 'saved_model70000.zip' # Нужно задать архив который проверяется

In [10]:
# Распаковываем архив с моделью, архив содержит папку `saved_model`
!unzip $MODEL_DOWNLOAD_PATH/$LAST_MODEL_ZIP -d '/content' 
%ls

Archive:  /content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/models/saved_model70000.zip
   creating: /content/saved_model/
  inflating: /content/saved_model/config.json  
  inflating: /content/saved_model/tokenizer_config.json  
  inflating: /content/saved_model/tokenizer.json  
  inflating: /content/saved_model/special_tokens_map.json  
  inflating: /content/saved_model/merges.txt  
  inflating: /content/saved_model/pytorch_model.bin  
  inflating: /content/saved_model/training_args.bin  
  inflating: /content/saved_model/vocab.json  
 extracting: /content/saved_model/added_tokens.json  
gdrive/  sample_data/  saved_model/


In [11]:
# Проверяем, что все файлы присутствуют
if not os.path.isdir(MODEL_PATH):
  raise Exception(f"Path '{MODEL_PATH}' not found!")

for fn in ['config.json', 'pytorch_model.bin', 'vocab.json', 'tokenizer_config.json', 'special_tokens_map.json', 'merges.txt']:
  if not os.path.isfile(os.path.join(MODEL_PATH, fn)):
    raise Exception(f'On path "{MODEL_PATH}" file "{fn}" not found!')

## Инициализируем библиотеки

In [13]:
import os
import random
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)

## Устанавливаем параметры модели

In [14]:
#@title Параметры модели

# Поле 'Max length tokens generate' должно быть больше 8, но меньше 256
max_length =  100#@param {type:"integer"} # 256
# def 3 Поле 'No repeat ngram size' должно быть больше или равно 1, но меньше или равно 10.
no_repeat_ngram_size = 2 #@param {type:"integer"}
# def 100 Поле 'Top K' должно быть больше или равно 1, но меньше или равно 500. Текущее значение:  100
top_k = 100 #@param {type:"integer"}
# def 0.9 Поле 'Top P' должно быть больше или равно 0.01, но меньше или равно 1.0.
top_p = 0.95 #@param {type:"number"} #0.9
# def 0.6 Поле 'Temperature' должно быть больше или равно 0.01, но меньше или равно 1.0. 
temperature = 0.6 #@param {type:"number"}
# def 5. Поле 'Num responses return' должно быть больше или равно 1, но меньше или равно 10. 
num_return = 5 #@param {type:"integer"}
# def True
do_sample = True  #@param {type:"boolean"}
# def True
is_always_use_length = True  #@param {type:"boolean"}
# Поле 'Length generate' должно принимать одно из следующих значений: [0, 1, 2, 3].
length_gen = 1  #@param {type:"integer"}
# def False. debug
log_debug = True  #@param {type:"boolean"}

params = { 
        "max_length": max_length,
        "no_repeat_ngram_size": no_repeat_ngram_size, 
        "top_k": top_k,                              
        "top_p": top_p,                              
        "temperature": temperature,                   
        "num_return_sequences": num_return,           
        "do_sample": do_sample,
        "device": "cuda" if torch.cuda.is_available() else "cpu", #'cuda:0', # 'cpu',
        "is_always_use_length": is_always_use_length,
        "length_generate": length_gen, 
        "log_debug": log_debug
    }
print(params)

{'max_length': 100, 'no_repeat_ngram_size': 2, 'top_k': 100, 'top_p': 0.95, 'temperature': 0.6, 'num_return_sequences': 5, 'do_sample': True, 'device': 'cpu', 'is_always_use_length': True, 'length_generate': 1, 'log_debug': True}


## Функции чатбота

In [15]:
# Функция определения длины диалога
def get_length_param(text: str, model_tokenizer) -> str:
    tokens_count = len(model_tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

In [16]:
# Генерация ответа
def generate(gpt3model, full_inputs, params, dialog_tokenizer): # inputs = [{'speaker': 0, 'text': 'Привет, как день прошел?'},{'speaker': 1, 'text': 'Хорошо, а у тебя как?'}]
  if len(full_inputs) > 5: # Ограничиваем кол-во истории в диалоге на 2 предыдущих + вопрос
    inputs = full_inputs[-5:]
  else:
    inputs = full_inputs
  inputs_text = ''
  for input_ in inputs:
    if params['is_always_use_length']:
        length_param = get_length_param(input_['text'], dialog_tokenizer)
    else:
        length_param = '-'
    inputs_text += f"|{input_['speaker']}|{length_param}|{input_['text']}"
  inputs_text += f"|1|{params['length_generate']}|"

  if params['log_debug']:
    print(f"        [debug] Затравка вопроса для модели: {inputs_text}")
  try:
    inputs_token_ids = dialog_tokenizer.encode(inputs_text, return_tensors='pt', add_special_tokens=False) # added add_special_tokens=False, 
    inputs_token_ids = dialog_tokenizer.encode(full_inputs[-1]['text'], return_tensors='pt', add_special_tokens=False) # added add_special_tokens=False, 
    # print('$$$$', len(inputs_token_ids),'!', inputs_token_ids, '$$$')
    inputs_token_ids = inputs_token_ids.to(params['device']) # Ошибка должны быть генерированы на одном устройстве
  except Exception as e:
    print('Ошибка', e)
  try:
    # ToDo make this asynchronous
    outputs_token_ids = gpt3model.generate(
        input_ids=inputs_token_ids,
        # inputs_token_ids,
        max_length=params['max_length'] + len(inputs_token_ids[0]),
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        do_sample=params['do_sample'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        temperature=params['temperature'],
        num_return_sequences=params['num_return_sequences'],
        device=params['device'],
        mask_token_id=dialog_tokenizer.mask_token_id,
        eos_token_id=dialog_tokenizer.eos_token_id,
        unk_token_id=dialog_tokenizer.unk_token_id,
        pad_token_id=dialog_tokenizer.pad_token_id,
    )
  except Exception as e:
    print(f"===> Error generate: ", e) # {str(e)}
    return {'inputs': '', 'outputs': '', 'status': False, 'msg': f"{str(e)}"}

  outputs = [dialog_tokenizer.decode(x, skip_special_tokens=True) for x in outputs_token_ids]
  outputs = [x.split('|')[-1] for x in outputs]

  return {'inputs': inputs, 'outputs': outputs, 'status': True, 'msg': ''}

## Загружаем модель

In [17]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
#MODEL_PATH = 'sberbank-ai/rugpt3small_based_on_gpt2' # Если проверяем работоспособность на базовой GPT, а не на обученной модели
dialog_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
dialog_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)




In [19]:
# вводить текстом
print('Введите quit для остановки')
if params['log_debug']:
  print(f"\n===> debug Params generate: {params}\n\n")
inputs = []
while True:
  user_input = input("USER:")
  if user_input == "quit":
    print ("Останавливаем диалог!")
    break
  inputs.append({'speaker': 0, 'text': user_input})
  response_data = generate(dialog_model, inputs, params, dialog_tokenizer) 
  if not response_data['status']:
    print("BOT (ошибка):", response_data['msg'])
  else:
    bot_resposnse = response_data['outputs'][0]
    inputs.append({'speaker': 1, 'text': bot_resposnse})
    print("\n>>>BOT:", bot_resposnse)
    variants_responses = response_data['outputs']
    if params['log_debug']:
      print("        [debug] Другие варианты ответа (можно выбирать отдельным алгоритмом наиболее подходящий) ===> :", variants_responses[1:])

Введите quit для остановки

===> debug Params generate: {'max_length': 100, 'no_repeat_ngram_size': 2, 'top_k': 100, 'top_p': 0.95, 'temperature': 0.6, 'num_return_sequences': 5, 'do_sample': True, 'device': 'cpu', 'is_always_use_length': True, 'length_generate': 1, 'log_debug': True}


USER:Привет, это проверка модели. Как дела?


Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


        [debug] Затравка вопроса для модели: |0|1|Привет, это проверка модели. Как дела?|1|1|

>>>BOT: Я бы хотела, чтобы вы мне сказали, что вы не можете быть счастливы, потому что у вас есть мама, которая вас не любит и не ценит. Вы говорите о себе и своих чувствах, а не о маме, которой вы можете сказать, как о своей маме. И если вы чувствуете,что мама вас любит, то вы говорите себе,
        [debug] other variants ===> : ['Я не знаю,какая бы я была в отношениях,если бы не встретила мужчину,который меня бы полюбил.У меня есть парень,и он очень хороший,а я не могу его принять.Я хочу его,но не понимаю,что мне делать.Мне хочется,чтобы он меня простил,чтоб я его не', 'Мне тоже не нравиться жить на своей территории. А я в этом не понимаю. И не знаю, как в таком случае жить. Мне в голову приходит только:', 'Это я тоже поняла. И не думаю,что он не понимает. Я ему и так говорила, но он стал злиться и говорить, почему ты так говоришь. Не пойму, если честно. Мы с ним разговаривали на эту тему, 

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


        [debug] Затравка вопроса для модели: |0|1|Привет, это проверка модели. Как дела?|1|3|Я бы хотела, чтобы вы мне сказали, что вы не можете быть счастливы, потому что у вас есть мама, которая вас не любит и не ценит. Вы говорите о себе и своих чувствах, а не о маме, которой вы можете сказать, как о своей маме. И если вы чувствуете,что мама вас любит, то вы говорите себе,|0|1|Конечно я счастлив. Это "психологический вопрос"?|1|1|

>>>BOT: Вот я и пытаюсь понять, может у него что то в голове не так, и он просто не хочет, чтобы я была с ним рядом и не могла просто взять и уйти. А я не могу уйти, я хочу с этим жить, хочу быть с человеком, которого люблю, но не хочу, чтоб он со мной спал
        [debug] other variants ===> : [' VadimCot писал(а): Вы хотите сказать,что если человек не хочет меняться,то он не может изменить себя?', 'Да, это так. Но он говорит, что у него нет никаких желаний и он хочет только одного, чтобы все было хорошо, но я не могу ему этого дать. Я не знаю, насколько

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


        [debug] Затравка вопроса для модели: |0|1|Привет, это проверка модели. Как дела?|1|3|Я бы хотела, чтобы вы мне сказали, что вы не можете быть счастливы, потому что у вас есть мама, которая вас не любит и не ценит. Вы говорите о себе и своих чувствах, а не о маме, которой вы можете сказать, как о своей маме. И если вы чувствуете,что мама вас любит, то вы говорите себе,|0|1|Конечно я счастлив. Это "психологический вопрос"?|1|3|Вот я и пытаюсь понять, может у него что то в голове не так, и он просто не хочет, чтобы я была с ним рядом и не могла просто взять и уйти. А я не могу уйти, я хочу с этим жить, хочу быть с человеком, которого люблю, но не хочу, чтоб он со мной спал|0|1|А вы говорите со мной или с кем? Такие странные комментарии|1|1|

>>>BOT: Вот и я о том же. Это не работает. Любовь, которая идет от души, когда ты ее видишь, знаешь, что она есть. А если не видишь - то это уже не любовь. И даже если любишь - это не значит,что она не твоя. Она не есть
        [debug] other v